to segment and recognise inner and outer boundaries of the organoids

In [33]:
import numpy as np
import napari
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage import filters
from skimage.filters import threshold_otsu, threshold_multiotsu
from skimage.measure import regionprops, label
from skimage.morphology import area_closing
# import cupy as cp
# from sys import getsizeof
# import pyclesperanto_prototype as cle
import cv2 as cv
import scipy
import pandas as pd
from matplotlib.pyplot import subplots
import os
import napari

In [3]:
image = imread('/Users/mandalt/A_PhD/ImageData/3D/20240710/splitChannels/DAPIC1-032_50cm_d15_shh_1x_05.tif')
image.shape

(17, 1024, 1024)

In [5]:
imagez7 = image[6,:,:]
# imagez7.imshow()

viewer = napari.Viewer()
viewer.add_image(imagez7, rgb=False, name='DAPI')

<Image layer 'DAPI' at 0x16c28a810>

In [8]:
# apply gaussian filter

imagez7_gf=filters.gaussian(imagez7, sigma=5)

viewer.add_image(imagez7_gf, rgb=False, name='DAPI+gf')

<Image layer 'DAPI+gf' at 0x12c07a390>

In [10]:

imagez7_th = threshold_otsu(imagez7_gf)
binary=imagez7_gf>imagez7_th
viewer.add_image(binary, rgb=False, name='threshold_binary', opacity=1.0, rendering='mip', blending='additive', colormap='gray')


<Image layer 'threshold_binary' at 0x1797e02c0>

In [16]:
l = label(binary)
out = (l==np.bincount(l.ravel())[1:].argmax()+1).astype(int)
out

viewer.add_image(out)


<Image layer 'out' at 0x3040361b0>

In [17]:
mask_boundary = skimage.segmentation.find_boundaries(out > 0, mode='outer')
viewer.add_image(mask_boundary)

<Image layer 'mask_boundary' at 0x30959b470>

In [18]:
dapi_gf_out = out*imagez7_gf

viewer.add_image(dapi_gf_out)

<Image layer 'dapi_gf_out' at 0x30954cf50>

In [28]:

imagez7_th2 = threshold_multiotsu(dapi_gf_out,classes=4)

imagez7_th2
# binary2=[dapi_gf_out<imagez7_th2[1] * dapi_gf_out>imagez7_th2[0]]

binary2=dapi_gf_out<imagez7_th2[1]
binary2 = binary2*out

binary2
viewer.add_image(binary2, rgb=False, name='threshold_binary', opacity=1.0, rendering='mip', blending='additive', colormap='gray')


<Image layer 'threshold_binary [1]' at 0x30df3dd00>

In [46]:
binary2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [45]:

mask2 = cv.inRange(binary2, 0.5, 1)
kernel_7=cv.getStructuringElement(cv.MORPH_ELLIPSE,(7,7))
holesFilled = cv.morphologyEx(mask2, cv.MORPH_CLOSE, kernel_7)


viewer.add_image(holesFilled)

<Image layer 'holesFilled' at 0x336c13020>

In [ ]:
#  plot settings
plt.gray()
plt.rcParams['font.size'] = 22
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.edgecolor'] = 'black'
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'